In [10]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset

In [3]:
dataset_train = load_dataset("stanfordnlp/imdb", split="train")
dataset_test = load_dataset("stanfordnlp/imdb", split="test")

def prompt_completion_preprocess(example):
    words = example['text'].split()
    prompt = ' '.join(words[:5])
    completion = ' '.join(words[5:])
    return {'prompt': prompt, 'completion': completion}



dataset_train = dataset_train.remove_columns(['label'])
dataset_test = dataset_test.map(prompt_completion_preprocess, remove_columns=['text', 'label'])


In [ ]:
model = 'Qwen/Qwen2-0.5B'

training_args = SFTConfig(
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    max_seq_length=512,
    output_dir = './qwen_imdb',
    report_to = 'wandb',
    weight_decay = 0.1,
    save_strategy="no"
)

trainer = SFTTrainer(
    model = model,
    args=training_args,
    train_dataset=dataset_train
)

trainer.train()
trainer.save_model("./qwen_imdb_final")  # ✅ manually save final model

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ooooaugust (ooooaugust-london-school-of-economics-and-political-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,3.245500
1000,3.204900
1500,3.162200
2000,2.794300
2500,2.755800
3000,2.756500
3500,2.462100
4000,2.374000
4500,2.375300


NameError: name 'tokenizer' is not defined

In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Path where you saved your fine-tuned model
model_dir = "./qwen_imdb_final"

# Load both model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model     = AutoModelForCausalLM.from_pretrained(model_dir)
model.push_to_hub("august66/qwen2-sft-imdb")
tokenizer.push_to_hub("august66/qwen2-sft-imdb")

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

ProxyError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/august66/qwen2-sft-imdb/commit/main (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))"), '(Request ID: ea91248a-92c6-45ac-a5dd-f5373bcc5071)')